In [1]:
from core.data_structures.hummingbot_database import HummingbotDatabase

import pandas as pd
import numpy as np
import os
import sys
import json
from hummingbot.connector.connector_base import TradeType
from decimal import Decimal
import plotly.graph_objects as go

from core.backtesting import BacktestingEngine
import warnings

warnings.filterwarnings("ignore")
root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)


In [2]:
from research_notebooks.xtreet_bb.xtreet_bt import XtreetBacktesting

db = HummingbotDatabase(db_name="la_maquinita-2024.sqlite", root_path=root_path)
backtesting_engine = BacktestingEngine(root_path=root_path, load_cached_data=True)
controllers = db.get_controller_data()
controller_ids = controllers["id"].tolist()
controller_id = controller_ids[5]

controller_executors = db.get_executors_from_controller_id(controller_id)
controller_config = json.loads(controllers.loc[controllers["id"] == controller_id, "config"].values[0])
max_timestamp = controller_executors["timestamp"].max()
min_timestamp = controller_executors["timestamp"].min()

In [3]:
from controllers.directional_trading.xtreet_bb import XtreetBBControllerConfig

config = XtreetBBControllerConfig(**controller_config)
connector = config.connector_name
trading_pair = config.trading_pair

In [4]:
start_time = backtesting_engine._dt_bt.backtesting_data_provider.candles_feeds[f"{connector}_{trading_pair}_1s"][
    "timestamp"].min() + 10
end_time = backtesting_engine._dt_bt.backtesting_data_provider.candles_feeds[f"{connector}_{trading_pair}_1s"][
    "timestamp"].max() - 10
backtesting_engine._dt_bt.backtesting_data_provider.start_time = start_time
backtesting_engine._dt_bt.backtesting_data_provider.end_time = end_time
backtesting_result = await backtesting_engine.run_backtesting(
    config=config,
    trade_cost=0.0007,
    start=int(start_time),
    end=int(end_time),
    backtesting_resolution="1s",
    backtester=XtreetBacktesting
)

AttributeError: 'int' object has no attribute 'get_controller_class'

In [ ]:
from hummingbot.strategy_v2.models.executors_info import ExecutorInfo

controller_executors["custom_info"] = controller_executors["custom_info"].apply(
    lambda x: json.loads(x) if isinstance(x, str) else x)
controller_executors["config"] = controller_executors["config"].apply(
    lambda x: json.loads(x) if isinstance(x, str) else x)
controller_executors_info = [ExecutorInfo(**controller) for controller in
                             controller_executors.to_dict(orient="records")]

In [ ]:
fig = backtesting_result.get_backtesting_figure()

In [ ]:
def add_executors_trace(executors, fig, row=1, col=1):
    for executor in executors:
        entry_time = pd.to_datetime(executor.timestamp, unit='s')
        entry_price = executor.custom_info["current_position_average_price"]
        exit_time = pd.to_datetime(executor.close_timestamp, unit='s')
        exit_price = executor.custom_info["close_price"]
        name = "Buy Executor" if executor.config.side == TradeType.BUY else "Sell Executor"

        if executor.filled_amount_quote == 0:
            fig.add_trace(go.Scatter(x=[entry_time, exit_time], y=[entry_price, entry_price], mode='lines', showlegend=False,
                                     line=dict(color='grey', width=2, dash="dash"), name=name), row=row, col=col)
        else:
            if executor.net_pnl_quote > Decimal(0):
                fig.add_trace(go.Scatter(x=[entry_time, exit_time], y=[entry_price, exit_price], mode='lines', showlegend=False,
                                         line=dict(color='green', width=2), name=name), row=row, col=col)
            else:
                fig.add_trace(go.Scatter(x=[entry_time, exit_time], y=[entry_price, exit_price], mode='lines', showlegend=False,
                                         line=dict(color='red', width=2), name=name), row=row, col=col)

    return fig


def get_pnl_trace(executors):
    pnl = [e.net_pnl_quote for e in executors]
    cum_pnl = np.cumsum(pnl)
    return go.Scatter(
        x=pd.to_datetime([e.close_timestamp for e in executors], unit="s"),
        y=cum_pnl,
        mode='lines',
        line=dict(color='gold', width=2),
        name='Cumulative PNL'
    )


def get_results_summary(results):
    net_pnl_quote = results["net_pnl_quote"]
    net_pnl_pct = results["net_pnl"]
    max_drawdown = results["max_drawdown_usd"]
    max_drawdown_pct = results["max_drawdown_pct"]
    total_volume = results["total_volume"]
    sharpe_ratio = results["sharpe_ratio"]
    profit_factor = results["profit_factor"]
    total_executors = results["total_executors"]
    accuracy_long = results["accuracy_long"]
    accuracy_short = results["accuracy_short"]
    take_profit = results["close_types"].get("TAKE_PROFIT", 0)
    stop_loss = results["close_types"].get("STOP_LOSS", 0)
    time_limit = results["close_types"].get("TIME_LIMIT", 0)
    trailing_stop = results["close_types"].get("TRAILING_STOP", 0)
    early_stop = results["close_types"].get("EARLY_STOP", 0)
    return f"""
Net PNL: ${net_pnl_quote:.2f} ({net_pnl_pct:.2f}%) | Max Drawdown: ${max_drawdown:.2f} ({max_drawdown_pct:.2f}%)
Total Volume ($): {total_volume:.2f} | Sharpe Ratio: {sharpe_ratio:.2f} | Profit Factor: {profit_factor:.2f}
Total Executors: {total_executors} | Accuracy Long: {accuracy_long:.2f} | Accuracy Short: {accuracy_short:.2f}
Close Types: Take Profit: {take_profit} | Stop Loss: {stop_loss} | Time Limit: {time_limit} |
             Trailing Stop: {trailing_stop} | Early Stop: {early_stop}
"""

In [ ]:
fig = add_executors_trace(controller_executors_info, fig)
fig.add_trace(get_pnl_trace(controller_executors_info), row=2, col=1)
bb_length = controller_config["bb_length"]
bb_std = controller_config["bb_std"]
# Add Bollinger Bands
bb_upper = f'BBU_{bb_length}_{bb_std}'
bb_lower = f'BBL_{bb_length}_{bb_std}'
resampled_features = backtesting_result.processed_data

In [ ]:

fig.add_trace(go.Scatter(x=resampled_features.index, y=resampled_features[bb_upper],
                         line=dict(color='rgba(173, 204, 255, 0.7)'),
                         name='BB Upper'), row=1, col=1)
fig.add_trace(go.Scatter(x=resampled_features.index, y=resampled_features[bb_lower],
                         line=dict(color='rgba(173, 204, 255, 0.7)'),
                         fill='tonexty', fillcolor='rgba(173, 204, 255, 0.1)',
                         name='BB Lower'), row=1, col=1)
results_live_controller = backtesting_engine._dt_bt.summarize_results(executors_info=controller_executors_info,
                                                                      total_amount_quote=controller_config[
                                                                          "total_amount_quote"])

print("Controller ID", controller_id)
print("Backtesting Results", backtesting_result.get_results_summary())
print("Live Bot Results", get_results_summary(results_live_controller))
fig.show()

In [ ]:
controllers